# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as F
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self, in_channels = 1, num_classes = 10):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 8,kernel_size = (3,3),stride=(1,1),padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2),stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels = 8,out_channels = 16,kernel_size = (3,3),stride = (1,1),padding = (1,1)) 
        self.conv3 = nn.Conv2d(in_channels = 16,out_channels = 32,kernel_size = (3,3),stride = (1,1),padding = (1,1))
        self.fc1 = nn.Linear(32*3*3,64)
        self.dropout = nn.Dropout(p = 0.25)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
      
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize   
        
        scores = model(images)
        loss = criterion(scores,labels)
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()

100%|██████████| 468/468 [01:06<00:00,  7.00it/s]


#### Q5:
Please print the training and testing accuracy.

In [5]:
correct = 0
total = 0 

for images, labels in train_loader:            
    test = Variable(images.view(128,1,28,28))
    outputs = model(test)
    predicted = torch.max(outputs.data, 1)[1]
    total += len(labels)
    correct += (predicted == labels).sum()       

train_accuracy = correct / float(total)
print('Training accuracy: %0.2f%%' % (train_accuracy*100))

correct = 0
total = 0 

for images, labels in test_loader:            
    test = Variable(images.view(128,1,28,28))
    outputs = model(test)
    predicted = torch.max(outputs.data, 1)[1]
    total += len(labels)
    correct += (predicted == labels).sum()       

test_accuracy = correct / float(total)
print('Testing accuracy: %0.2f%%' % (test_accuracy*100))

Training accuracy: 98.48%
Testing accuracy: 98.47%
